In [ ]:
#include <iostream>
#include <vector>
#include <queue>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <thread>
#include <chrono>

using namespace std;

struct Celda {
    bool visitada = false;
    bool paredes[4] = {true,true,true,true}; 
    bool enCamino = false;
    bool caminoCorto = false;
};

int dx[4] = {0,1,0,-1};
int dy[4] = {-1,0,1,0};

int filas = 0, columnas = 0;
vector<vector<Celda>> laberinto;

bool esValido(int x,int y){ 
    return x>=0 && x<columnas && y>=0 && y<filas; 
}

void removerPared(Celda &a, Celda &b, int dir){
    a.paredes[dir] = false;
    b.paredes[(dir+2)%4] = false;
}

void imprimirLaberinto(){
    system("CLS");
    for(int y=0;y<filas;y++){
        for(int x=0;x<columnas;x++) cout << "+" << (laberinto[y][x].paredes[0]?"---":"   ");
        cout << "+\n";
        for(int x=0;x<columnas;x++){
            cout << (laberinto[y][x].paredes[3]?"|":" ");
            if(y==0 && x==0) cout << " S ";
            else if(y==filas-1 && x==columnas-1) cout << " E ";
            else if(laberinto[y][x].caminoCorto) cout << " O ";
            else if(laberinto[y][x].enCamino) cout << " X ";
            else cout << "   ";
        }
        cout << "|\n";
    }
    for(int x=0;x<columnas;x++) cout << "+---";
    cout << "+\n";
}

void generarLaberintoDFS(int x,int y){
    laberinto[y][x].visitada=true;
    laberinto[y][x].enCamino=true;
    imprimirLaberinto();
    this_thread::sleep_for(chrono::milliseconds(100));

    vector<int> dirs={0,1,2,3};
    random_shuffle(dirs.begin(),dirs.end());

    for(int dir:dirs){
        int nx=x+dx[dir], ny=y+dy[dir];
        if(esValido(nx,ny) && !laberinto[ny][nx].visitada){
            removerPared(laberinto[y][x],laberinto[ny][nx],dir);
            generarLaberintoDFS(nx,ny);
        }
    }

    laberinto[y][x].enCamino=false;
    imprimirLaberinto();
    this_thread::sleep_for(chrono::milliseconds(50));
}

// Reconstruir camino más corto desde S a E con BFS
void caminoMasCorto(){
    queue<pair<int,int>> q;
    vector<vector<pair<int,int>>> padre(filas,vector<pair<int,int>>(columnas,{-1,-1}));
    q.push({0,0});
    vector<vector<bool>> visit(filas,vector<bool>(columnas,false));
    visit[0][0]=true;

    while(!q.empty()){
        auto [x,y]=q.front(); q.pop();
        if(x==columnas-1 && y==filas-1) break;
        for(int dir=0;dir<4;dir++){
            int nx=x+dx[dir], ny=y+dy[dir];
            if(esValido(nx,ny) && !visit[ny][nx] && !laberinto[y][x].paredes[dir]){
                visit[ny][nx]=true;
                padre[ny][nx]={x,y};
                q.push({nx,ny});
            }
        }
    }

    int x=columnas-1, y=filas-1;
    while(!(x==0 && y==0)){
        laberinto[y][x].caminoCorto=true;
        auto p=padre[y][x];
        x=p.first; y=p.second;
        imprimirLaberinto();
        this_thread::sleep_for(chrono::milliseconds(50));
    }
    laberinto[0][0].caminoCorto=true;
}

int main(){
    srand(time(0));
    char opcion;

    do{
        cout << "\nOpciones:\n";
        cout << "1. Generar laberinto\n";
        cout << "2. Resolver laberinto (camino más corto)\n";
        cout << "3. Salir\n";
        cout << "Opcion: ";
        cin >> opcion;

        if(opcion=='1'){
            cout << "\nIngrese filas: "; cin >> filas;
            cout << "Ingrese columnas: "; cin >> columnas;
            laberinto=vector<vector<Celda>>(filas,vector<Celda>(columnas));

            cout << "\nGenerando laberinto de " << filas << " x " << columnas << "...\n";
            generarLaberintoDFS(0,0);
            cout << "¡Laberinto generado!\n";
            imprimirLaberinto();
        }
        else if(opcion=='2'){
            if(filas==0 || columnas==0){
                cout << "Primero genera un laberinto.\n";
                continue;
            }
            cout << "Mostrando camino más corto...\n";
            caminoMasCorto();
            cout << "¡Laberinto resuelto!\n";
            imprimirLaberinto();
        }
    }while(opcion!='3');

    cout << "Saliendo...\n";
    return 0;
}